In [1]:
import api_nyt as nyt
import api_wikipedia as wiki
import manage_articles as mng
import statistics
import datetime
import pickle
import timeseries
import TextPreprocessing as txt
from datetime import date
import graphics
import matplotlib.pyplot as plt
import matching
import time
import word
import random
import numpy

articles = mng.load_articles("nyt2019.json")

In [2]:
#headlines = mng.getWordCounts(articles,'headline')
#with open("wordCount2019.txt", "wb") as fp:
#    pickle.dump(headlines, fp)
#top = mng.getTopWordsForWeek(headlines)
#distinctWords = mng.getDistinctWords(headlines)

headlines = []
with open("wordCount2019.txt", "rb") as fp:
    headlines = pickle.load(fp)
distinctWords = mng.getDistinctWords(headlines)

countsPerWeek = []
for w in distinctWords:
    countsPerWeek.append((w,mng.getCountPerWeek(headlines,w)))

words = []
for c in countsPerWeek:
    words.append(word.Word(c[0],ts_articles=timeseries.Timeseries(c[1])))
words = sorted(words, key=lambda x: sum(x.ts_articles.getCounts()), reverse=True)

In [3]:
top_amount = 150

keywords = []
for i in range(top_amount):
    keywords.append(words[i].keyword)

m = matching.groupmatch(keywords, articles)

for key in m.keys():
    #Following line from 
    #https://stackoverflow.com/questions/7125467/find-object-in-list-that-has-attribute-equal-to-some-value-that-meets-any-condi
    word = next((x for x in words if x.keyword == key), None)
    query = m[key]['query']
    page = m[key]['link'][1]
    
    wiki_counts = wiki.get_counts(page, word.ts_articles.getStartDate(), word.ts_articles.getEndDate(),"en")
    if wiki_counts is not None:
        word.coocKeywords = query
        word.wikipediaSite = page
        word.ts_wiki = timeseries.parseWikipediaCounts(wiki_counts)
    


404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found


In [4]:
words_analyze = [x for x in words if x.wikipediaSite != ""]

corr = []
corr_rand = []

a = [x for x in range(len(words_analyze))] 
b = [x for x in range(len(words_analyze))] 
random.shuffle(b)

#remove words
for w in words_analyze:
    ts_a, ts_w = timeseries.alignTimeseries(w.ts_articles,w.ts_wiki)
    if len(ts_w.getCounts())<2 or len(ts_a.getCounts())<2:
        words_analyze.remove(w)
    else:
        corr.append(statistics.getCorrelation(ts_a.getCounts(),ts_w.getCounts()))


In [5]:
import numpy
print(numpy.mean(corr))

0.24973154623852784
